In [ ]:
import collection

In [2]:
#path_txt = "corpus_txt/"

path_txt = "grenzbote_plain/*/"

#path_txt = "drama/"

doclist_txt = collection.create_document_list(path_txt)

doc_labels = list(collection.get_labels(doclist_txt))

corpus_txt = collection.read_from_txt(doclist_txt)

doc_tokens = list(collection.tokenizer(corpus_txt))

id_types, doc_dict, doc_ids = collection.create_large_TF_matrix(doc_labels, doc_tokens)

termdoc_matrix = {v : k for k, v in id_types.items()}

id_docs = {v : k for k, v in doc_ids.items()}

In [3]:
largecounter = collection.create_large_counter(doc_labels, doc_tokens, termdoc_matrix)

largecounter = {id_docs[key] : value for key, value in largecounter.items()}

In [27]:
len(largecounter)

410

In [5]:
#len(sparse_index)

In [6]:
#len(pd.DataFrame(np.zeros((len(sparse_index.get_level_values('token_id')),1), dtype = int), index = sparse_index))

In [7]:
import pandas as pd
import numpy as np
def populate_two(sparse_index, largecounter):
    
    #sparse_df_filled_test = pd.Series(index=sparse_index).fillna(int(0))
    sparse_df_filled_test = pd.DataFrame(np.zeros((len(sparse_index), 1), dtype = int), index = sparse_index)
    
    
    index_iterator = sparse_index.groupby(sparse_index.get_level_values('doc_id'))
    
    for doc_id in range(len(sparse_index.levels[0])):
        for token_id in [val[1] for val in index_iterator[doc_id]]:
            
            sparse_df_filled_test.set_value((doc_id, token_id), 0, int(largecounter[doc_id][token_id]))
            
    return sparse_df_filled_test
    

In [8]:
sparse_index = collection.create_sparse_index(largecounter)

In [9]:
sparse_df = populate_two(sparse_index, largecounter)

In [10]:
#sparse_index = collection.create_sparse_index(largecounter)

#sparse_df = collection.populate_two(sparse_index, largecounter)

#sparse_df.to_csv('sparse_drama.mm', sep = ' ')

In [11]:
sparse_df = sparse_df.sortlevel()

In [12]:
max(sparse_df[0])

444

In [13]:
max(sparse_df.index.get_level_values("doc_id"))

409

In [14]:
max(sparse_df.index.get_level_values("token_id"))

99175

In [15]:
sum(sparse_df[0])

1286350

In [16]:


num_docs = max(sparse_df.index.get_level_values("doc_id"))
num_types = max(sparse_df.index.get_level_values("token_id"))
sum_counts = sum(sparse_df[0])

header_string = str(num_docs) + " " + str(num_types) + " " + str(sum_counts) + "\n"

with open("gb_plain.mm", 'a', encoding = "utf-8") as f:
    f.write("%%MatrixMarket matrix coordinate real general\n")
    #f.write(header_string)
    sparse_df.to_csv( f, sep = ' ', header = None)



In [17]:
import gensim

In [18]:
corpusPath = "gb_plain.mm"

mm = gensim.corpora.MmCorpus(corpusPath)

In [19]:
#in case you're only loading the corpus - build dict first

import pickle

pickle.dump( id_types, open( "gb_plain.dictionary", "wb" ) )

In [20]:
id_types = pickle.load(open("gb_plain.dictionary", 'rb'))

In [21]:
#mm = gensim.corpora.MmCorpus("gb_all.mm")

In [22]:
#id_types = pickle.load(open("gb_all.dictionary", 'rb'))

In [23]:
model = gensim.models.LdaModel(corpus=mm, id2word=id_types, num_topics=60, alpha = "symmetric", passes = 10)

In [24]:
topic_nr_x = model.get_topic_terms(10)

[id_types[i[0]] for i in topic_nr_x]

['thronwechsel',
 'lliot',
 'directionsmitglieder',
 'oswald',
 'künstlerischer',
 'schießplatz',
 'vollkommenheit',
 'wandertrieb',
 'nichterstcllen',
 'nhetz']

In [25]:
topics = model.show_topics(num_topics = 60)

In [26]:
topics

[(0,
  '0.000*wahlvrüfungcn + 0.000*werd + 0.000*geschäftsfreunden + 0.000*überzusiedeln + 0.000*evvivas + 0.000*cyklen + 0.000*segensreichste + 0.000*treitzschkes + 0.000*buckle + 0.000*517'),
 (1,
  '0.000*machtvollkommenheit + 0.000*emphatischcrem + 0.000*saale + 0.000*ciellen + 0.000*donizetti + 0.000*culturverhältnissen + 0.000*krähet + 0.000*angewandten + 0.000*formulartöpfc + 0.000*professors'),
 (2,
  '0.000*vorsorge + 0.000*zwanglos + 0.000*back + 0.000*harmloser + 0.000*zustimmenden + 0.000*untergang + 0.000*tacten + 0.000*1665 + 0.000*unmündige + 0.000*dalismus'),
 (3,
  '0.000*starb + 0.000*labedoyöre + 0.000*tändcrn + 0.000*jahrelang + 0.000*staatswissenschaften + 0.000*lebenskräftigen + 0.000*lesevereins + 0.000*generalsecretär + 0.000*pettenkofer + 0.000*fischart'),
 (4,
  '0.000*beimischen + 0.000*kernbevölkcrung + 0.000*überspannt + 0.000*verbessertes + 0.000*slavomanie + 0.000*gleichwichtige + 0.000*funkeln + 0.000*herbstabenden + 0.000*diplomat + 0.000*willkommen'),
